In [1]:
import numpy as np
import torch

from helper_functions import *

In [43]:
dm = load_tsdataset("WISDM", "./datasets/", 256, 8, 48, 1, True, None, [30, 31, 32, 33, 34, 35], True)

Loaded dataset WISDM with a total of 1096480 observations for window size 48
Using 533566 observations for training and 154375 observations for validation and test


In [3]:
dm.stsds.SCS[dm.stsds.indices[dm.ds_train.indices]].unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([ 88321, 128232,  50091,  40497, 108362, 118417]))

In [4]:
dm.stsds.SCS[dm.stsds.indices[dm.ds_test.indices]].unique(return_counts=True)

(tensor([0, 1, 2, 3, 4, 5]),
 tensor([12077, 38575,  9828,  7857, 14484, 71722]))

In [45]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "ts", "cnn", "mlp", "cls", "test", 20, 64, 2, 0.001)

Input shape:  torch.Size([1, 3, 48])
Latent shape:  torch.Size([1, 80, 12])
[tensor(960), 64, 64, 6]


In [47]:
model.encoder.summary()

AttributeError: 'CNN_TS' object has no attribute 'summary'

In [6]:
from pytorch_lightning import Trainer

In [7]:
from s3ts.api.nets.methods import create_model_from_DM, train_model


In [48]:
train_model(dm, model, 5)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name        | Type                 | Params
------------------------------------------------------
0  | encoder     | CNN_TS               | 12.9 K
1  | decoder     | MultiLayerPerceptron | 66.1 K
2  | flatten     | Flatten              | 0     
3  | softmax     | Softmax              | 0     
4  | train_acc   | MulticlassAccuracy   | 0     
5  | train_f1    | MulticlassF1Score    | 0     
6  | train_auroc | MulticlassAUROC      | 0     
7  | val_acc     | MulticlassAccuracy   | 0     
8  | val_f1      | MulticlassF1Score    | 0     
9  | val_auroc   | MulticlassAUROC      | 0     
10 | test_acc    | MulticlassAccuracy   | 0     
11 | test_f1     | MulticlassF1Score    | 0     
12 | test_auroc  | MulticlassAUROC      | 0     
------------------------------------------------------
79.0 K    Trainable params
0      

Epoch 0:  18%|█▊        | 373/2085 [00:07<00:33, 50.62it/s, v_num=3, train_loss_step=0.593]

IsADirectoryError: [Errno 21] Is a directory: '/home/mbikandi/Documents/s3ts'

In [9]:
tr = Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
tr.validate(model, dm)

Validation DataLoader 0: 100%|██████████| 604/604 [00:03<00:00, 171.14it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.7493577599525452     │
│         val_auroc         │    0.02436981350183487    │
│          val_f1           │    0.7493577599525452     │
│         val_loss          │    0.6932730674743652     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.6932730674743652,
  'val_acc': 0.7493577599525452,
  'val_f1': 0.7493577599525452,
  'val_auroc': 0.02436981350183487}]

In [11]:
dm.ds_test[0]["label"]

tensor(5)

In [12]:
dm.ds_test[200]["series"].max()

tensor(0.3793)

In [13]:
dm.ds_test[0]["series"].unsqueeze(0).shape

torch.Size([1, 3, 20])

In [14]:
model.eval()

WrapperModel(
  (encoder): SimpleCNN_TS(
    (model): Sequential(
      (0): Conv1d(3, 8, kernel_size=(3,), stride=(1,))
      (1): ReLU()
      (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv1d(8, 16, kernel_size=(4,), stride=(1,))
      (4): ReLU()
      (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
      (7): ReLU()
    )
  )
  (decoder): MultiLayerPerceptron(
    (fcn_layer_0): Linear(in_features=32, out_features=16, bias=True)
    (act_layer_0): ReLU()
    (fcn_out): Linear(in_features=16, out_features=6, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=None)
  (train_acc): MulticlassAccuracy()
  (train_f1): MulticlassF1Score()
  (train_auroc): MulticlassAUROC()
  (val_acc): MulticlassAccuracy()
  (val_f1): MulticlassF1Score()
  (val_auroc): MulticlassAUROC()
  (test_acc): MulticlassAccuracy()
  (test_f1): M

In [15]:
n=15000
print(dm.stsds[dm.ds_test.indices[n]][1])
print(dm.ds_test[n]["label"])

model(dm.ds_test[n]["series"].unsqueeze(0))[0]

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
tensor(5)


tensor([1.8617e-02, 3.5694e-03, 6.3668e-05, 2.0013e-07, 2.4172e-01, 7.3603e-01],
       grad_fn=<SelectBackward0>)

In [16]:
out = torch.zeros((len(dm.ds_test), 6))
lb = torch.zeros(len(dm.ds_test))
for i in range(len(dm.ds_test)):
    d = dm.ds_test[i]
    out[i, :] = model.logits(d["series"].unsqueeze(0))[0]
    lb[i] = d["label"]

In [17]:
import torchmetrics as tm

In [33]:
confm = tm.ConfusionMatrix(task="multiclass", num_classes=6)

In [34]:
acc = tm.Accuracy(task="multiclass", num_classes=6, average="macro")

In [35]:
p = tm.Precision(task="multiclass", num_classes=6, average="macro")
r = tm.Recall(task="multiclass", num_classes=6, average="macro")

In [36]:
auroc = tm.AUROC(task="multiclass", num_classes=6, average="macro")

In [37]:
lb.unique(return_counts=True)

(tensor([0., 1., 2., 3., 4., 5.]),
 tensor([12077, 38577,  9826,  7854, 14484, 71725]))

In [38]:
confm(out, lb)

tensor([[ 5505,    58,    26,    86,  3914,  2488],
        [ 1633, 32394,     1,     2,  2518,  2029],
        [   12,     0,  8505,     2,  1307,     0],
        [    4,     0,    14,  4288,  3538,    10],
        [ 1553,    24,    25,    61,  9205,  3616],
        [ 4198,   202,    51,   354, 11009, 55911]])

In [39]:
auroc(out, lb.long())

tensor(0.9392)

In [40]:
acc(out, lb.long())

tensor(0.6870)

In [41]:
p(out, lb.long())

tensor(0.7440)

In [42]:
r(out, lb.long())

tensor(0.6870)